In [1]:
import sys
import torch
import time
import matplotlib.pyplot as plt
import copy

from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from itertools import islice
from os import makedirs, path, getcwd
import numpy as np

from utils import get_dataset, num_batches_per_epoch
from options import args_parser
from update import test_inference
from models import get_model, get_optimizer
from argparse import ArgumentParser
from torch.utils.data import DataLoader, Dataset

from update import ClientShard, test_inference

from tqdm import tqdm
from os import path, getcwd, makedirs
from tensorboardX import SummaryWriter

sys.path.insert(0, path.join(getcwd(), "..", ".."))

from options import args_parser
#from update import ClientShard, test_inference
from models import get_model, VGG, CNNMnist
from utils import (
    get_dataset,
    average_weights,
    exp_details,
)

#MONAI Imports
import os
import shutil
import tempfile
import time
import matplotlib.pyplot as plt
import numpy as np
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader, decollate_batch,ArrayDataset, GridPatchDataset
from monai.handlers.utils import from_engine
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.networks.nets import SegResNet

from torch.utils.tensorboard import SummaryWriter
import monai
from monai.transforms import (
    Activations,
    Activationsd,
    AddChannel,
    AsDiscrete,
    AsDiscreted,
    AsChannelFirst,
    Compose,
    ConvertToMultiChannelBasedOnBratsClassesd,
    Invertd,
    LoadImaged,
    LabelToMask,
    LoadImage,
    MapTransform,
    NormalizeIntensity,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    RandSpatialCrop,
    Resize,
    RandFlip,
    RandRotate,
    Spacing,
    Spacingd,
    ScaleIntensity, 
    SpatialCrop,
    ToTensor,
    EnsureChannelFirstd,
    EnsureTyped,
    EnsureType,
    Resized
)
from monai.utils import set_determinism
from natsort import natsorted
import torch

from glob import glob
import os.path
import nibabel as nib
from nibabel import load, save, Nifti1Image
%matplotlib notebook


print_config()


MONAI version: 0.8.0
Numpy version: 1.20.3
Pytorch version: 1.10.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.7
Nibabel version: 3.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 8.4.0
Tensorboard version: 2.7.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.11.1
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: 1.3.4
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
parser = ArgumentParser('FL experiments parser')

In [3]:
# federated arguments (Notation for the arguments followed from paper)
parser.add_argument('--epochs', type=int, default=10,
                    help="number of rounds of training")
parser.add_argument('--num_users', type=int, default=100,
                    help="number of users: K")
parser.add_argument('--test_frac', type=float, default=0.1,
                    help='the fraction of data to go into test split.')
parser.add_argument('--frac', type=float, default=0.1,
                    help='the fraction of clients: C')
parser.add_argument('--local_ep', type=int, default=10,
                    help="the number of local epochs: E")
parser.add_argument('--local_bs', type=int, default=8,
                    help="local batch size: B")
parser.add_argument('--lr', type=float, default=0.01,
                    help='learning rate')
parser.add_argument('--momentum', type=float, default=0.5,
                    help='SGD momentum (default: 0.5)')

# model arguments
parser.add_argument('--task', type=str, default='cv', help='task name')
parser.add_argument('--model', type=str, default='cnn', help='model name')
parser.add_argument('--kernel_num', type=int, default=9,
                    help='number of each kind of kernel')
parser.add_argument('--kernel_sizes', type=str, default='3,4,5',
                    help='comma-separated kernel size to \
                    use for convolution')
parser.add_argument('--num_channels', type=int, default=3, help="number \
                    of channels of imgs")
parser.add_argument('--norm', type=str, default='batch_norm',
                    help="batch_norm, layer_norm, or None")
parser.add_argument('--num_filters', type=int, default=32,
                    help="number of filters for conv nets -- 32 for \
                    mini-imagenet, 64 for omiglot.")
parser.add_argument('--max_pool', type=str, default='True',
                    help="Whether use max pooling rather than \
                    strided convolutions")

# other arguments
parser.add_argument('--dataset', type=str, default='cifar', help="name \
                    of dataset")
parser.add_argument('--num_classes', type=int, default=10, help="number \
                    of classes")
parser.add_argument('--gpu', default=None, help="To use cuda, set \
                    to a specific GPU ID. Default set to use CPU.")
parser.add_argument('--optimizer', type=str, default='sgd', help="type \
                    of optimizer")
parser.add_argument('--iid', type=int, default=1,
                    help='Default set to IID. Set to 0 for non-IID.')
parser.add_argument('--unequal', type=int, default=0,
                    help='whether to use unequal data splits for  \
                    non-i.i.d setting (use 0 for equal splits)')
parser.add_argument('--stopping_rounds', type=int, default=10,
                    help='rounds of early stopping')
parser.add_argument('--ROOT_DATA', type=str, default='./data/',
                    help='Root data dyrectory ')

parser.add_argument('--verbose', type=int, default=1, help='verbose')
parser.add_argument('--seed', type=int, default=1, help='random seed')



_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='random seed', metavar=None)

In [4]:
LIEU = 'sebastian_insel'

if LIEU=='sebastian_insel':
    root_exp  = '/home/sebastian/experiments/fedem/'
    root_data = '/media/sebastian/data/ASAP/BRATS_2019_ubelix/center_wise/'
    brats_nifti__dir_paths = natsorted(glob(root_data+"**/**/"))

if LIEU=='sebastian_laptop':
    root_exp  = '/Users/sebastianotalora/work/postdoc/federated_learning/fedem/'
    root_data = '/Users/sebastianotalora/work/postdoc/federated_learning/data/'
    brats_nifti__dir_paths = natsorted(glob(root_data+'brats/'"**/**/"))
len(brats_nifti__dir_paths)#Should be 259


259

In [5]:
args = parser.parse_args(["--task", "cv", "--dataset", "brats", "--epochs", "2", "--num_users", "4", 
                          "--ROOT_DATA", root_data, "--gpu", "True",
                          "--frac", "2", "--local_ep", "5"])

In [6]:
args.ROOT_DATA

'/media/sebastian/data/ASAP/BRATS_2019_ubelix/center_wise/'

In [7]:
device =  'cuda' if args.gpu else 'cpu'
path_project = path.abspath('..')
logger = SummaryWriter('./logs')

In [8]:
device

'cuda'

In [9]:
print(vars(args))

{'epochs': 2, 'num_users': 4, 'test_frac': 0.1, 'frac': 2.0, 'local_ep': 5, 'local_bs': 8, 'lr': 0.01, 'momentum': 0.5, 'task': 'cv', 'model': 'cnn', 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channels': 3, 'norm': 'batch_norm', 'num_filters': 32, 'max_pool': 'True', 'dataset': 'brats', 'num_classes': 10, 'gpu': 'True', 'optimizer': 'sgd', 'iid': 1, 'unequal': 0, 'stopping_rounds': 10, 'ROOT_DATA': '/media/sebastian/data/ASAP/BRATS_2019_ubelix/center_wise/', 'verbose': 1, 'seed': 1}


In [10]:
# Load dataset
train_dataset, test_dataset, user_groups, user_groups_test = get_dataset(args)

# Load monai segmentation model.
global_model = get_model(args=args, img_size=train_dataset[0][0].shape)


259
loading BraTS data from /media/sebastian/data/ASAP/BRATS_2019_ubelix/center_wise//
180 28 51
Loading BraTS segmentation Network


In [11]:
global_model

SegResNet(
  (act): ReLU(inplace=True)
  (convInit): Convolution(
    (conv): Conv3d(4, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
  )
  (down_layers): ModuleList(
    (0): Sequential(
      (0): Identity()
      (1): ResBlock(
        (norm1): GroupNorm(8, 16, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 16, eps=1e-05, affine=True)
        (relu): ReLU(inplace=True)
        (conv1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
        (conv2): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
    )
    (1): Sequential(
      (0): Convolution(
        (conv): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (1): ResBlock(
        (norm1): GroupNorm(8, 32, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 32, eps=1e-05, affi

In [12]:
### Start of Federated learning. {{{
# Set the model to train and send it to device.
global_model.to(device)
global_model.train()
#print(global_model)
# copy weights
global_weights = global_model.state_dict()
# Training
train_loss, train_dice = [], []
print_every = 1
exp_details(args)


Experimental details:
    Task      : cv
    Model     : cnn
    Optimizer : sgd
    Learning  : 0.01
    Global Rounds   : 2

    Federated parameters:
    IID
    Fraction of users  : 2.0
    Local Batch size   : 8
    Local Epochs       : 5



In [13]:
args.local_bs = 1
args.epochs   = 1

In [14]:
idxs_users = np.array([0,1,2,3])

In [16]:
args.num_users

4

In [17]:
start_time = time.time()
# Load monai segmentation model.
global_model = get_model(args=args, img_size=train_dataset[0][0].shape)


for epoch in range(args.epochs):
    local_weights, local_losses = [], []
    global_model.train()
    # Randomly sample a fraction of clients and retrieve their ids.
    user_frac = 2#max(int(args.frac * args.num_users), 1)
    #idxs_users = np.random.choice(range(args.num_users), user_frac, replace=False)
    idxs_users = np.array([0,1,2,3]) # In the BraTS case we train with ALL the clients
    
    for hidden_client_idx, idx in enumerate(idxs_users):
        client_shard = ClientShard(args=args,
                                   client_idx=hidden_client_idx,
                                   dataset=train_dataset,
                                   idxs=user_groups[idx],
                                   logger=logger,
                                   device=device)
        updated_local_model, loss = client_shard.update_weights(model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(updated_local_model))
        local_losses.append(copy.deepcopy(loss))
        
    # Calculate averaged model weights from all the client trained models.
    global_weights = average_weights(local_weights)
    # Update global weights with the averaged model weights.
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)
    
    # Calculate avg training DICE over all users at every epoch
    list_dice, list_loss = [], []
    global_model.eval()
    # Remove trained clients from heldout evaluation.
    heldout_clients = list(range(args.num_users))#We evaluate in the test set of each client

    for heldout_client_idx, idx in tqdm(enumerate(heldout_clients),
                                        desc='Evaluating: Hidden client num:',
                                        total=len(heldout_clients)):
        client_shard = ClientShard(args=args,
                                   client_idx=heldout_client_idx,
                                   dataset=train_dataset,
                                   idxs=user_groups[idx],
                                   logger=logger,
                                   device=device)
        loss, dice_client = client_shard.inference(model=global_model)
        list_dice.append(dice_client)
        list_loss.append(loss)
    train_dice.append(sum(list_dice) / len(list_dice))

    # print global training loss after every 'i' rounds
    if (epoch + 1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch + 1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train DICE: {:.2f}% \n'.format(100 * train_dice[-1]))

Loading BraTS segmentation Network


/home/sebastian/anaconda3/envs/fed/lib/python3.9/site-packages/monai/transforms/post/array.py:182: UserWarning: `threshold_values=True/False` is deprecated, please use `threshold=value` instead.
  warnings.warn("`threshold_values=True/False` is deprecated, please use `threshold=value` instead.")


| Global Round : 0 | Hidden client num : 0 | Local Epoch : 4 | [10/12 (83%)]	Loss: 0.928311

/home/sebastian/anaconda3/envs/fed/lib/python3.9/site-packages/monai/transforms/post/array.py:182: UserWarning: `threshold_values=True/False` is deprecated, please use `threshold=value` instead.
  warnings.warn("`threshold_values=True/False` is deprecated, please use `threshold=value` instead.")


| Global Round : 0 | Hidden client num : 1 | Local Epoch : 4 | [80/83 (96%)]	Loss: 0.972865

...

| Global Round : 0 | Hidden client num : 2 | Local Epoch : 4 | [60/65 (92%)]	Loss: 0.958628

/home/sebastian/anaconda3/envs/fed/lib/python3.9/site-packages/monai/transforms/post/array.py:182: UserWarning: `threshold_values=True/False` is deprecated, please use `threshold=value` instead.
  warnings.warn("`threshold_values=True/False` is deprecated, please use `threshold=value` instead.")


| Global Round : 0 | Hidden client num : 3 | Local Epoch : 4 | [0/4 (0%)]	Loss: 0.935884

Evaluating: Hidden client num::   0%|                                                                                                                                                       | 0/4 [00:00<?, ?it/s]/home/sebastian/anaconda3/envs/fed/lib/python3.9/site-packages/monai/transforms/post/array.py:182: UserWarning: `threshold_values=True/False` is deprecated, please use `threshold=value` instead.
  warnings.warn("`threshold_values=True/False` is deprecated, please use `threshold=value` instead.")
Evaluating: Hidden client num::   0%|                                                                                                                                                       | 0/4 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (903168) must match the size of tensor b (72) at non-singleton dimension 4

In [21]:
i = 0
for item in client_shard.testloader:
    print(i)
    i+=1

0


In [48]:
for heldout_client_idx, idx in tqdm(enumerate(heldout_clients),
                                        desc='Evaluating: Hidden client num:',
                                        total=len(heldout_clients)):
    print(heldout_client_idx, idx)

Evaluating: Hidden client num:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 6227.62it/s]

0 0
1 1
2 2
3 3


In [ ]:
# Test inference after completion of training
test_dice, test_loss = test_inference(args=args, model=global_model, test_dataset=test_dataset, device=device)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train DICE: {:.2f}%".format(100*train_dice[-1]))
print("|---- Test DICE: {:.2f}%".format(100*test_dice))
### }}} End of Federated learning.

